In [1]:
import numpy as np
import pandas as pd 
import glob, re
from collections import Counter
from nltk.tokenize import wordpunct_tokenize as tokenize
import string, random, os

In [5]:
docs = []
for ind, filename in enumerate(glob.glob("../data/whole-data/*")):
    if ind % 10000 == 0:
        print(ind, 'docs read')
    fd = open(filename, 'r')
    text = fd.read() 
    docs.append(text)
    fd.close()


0 docs read
10000 docs read
20000 docs read
30000 docs read
40000 docs read
50000 docs read
60000 docs read
70000 docs read
80000 docs read
90000 docs read
100000 docs read
110000 docs read
120000 docs read
130000 docs read
140000 docs read
150000 docs read
160000 docs read
170000 docs read
180000 docs read
190000 docs read


ვიკიპედიას კორპუსში ბევრი არასასურველი სიტყვაა და ჯობს მინიმალურამდე დავიყვანოთ მათი რაოდენობა. ამისთვის, თითოეულ სტატიაში ვითვლით რამდენი "კარგი" და "ცუდი" სიტყვაა და მათი სხვაობის მიხედვით ტოპ 50 000 სტატიას ვარჩევთ. ამ მიდგომით შეგვრჩება 50 000 სტატია, რომელშიც ყველაზე მეტი სასურველი სიტყვაა და თან თავიდან ავირიდებთ მცირე ზომის სტატიებს. 

In [6]:
tokenized = [tokenize(doc) for doc in docs]

In [7]:
# თუ სიტყვა შეიცავს ერთ სიმბოლოს მაინც, რომელიც ქვედა სიმბოლოების სიაში არ შედის, ესე იგი არასასურველი სიტყვაა.
punct = '!"#%&+,-./:;=?()²' + "'"
letters = 'აბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰ'
numbers = '0123456789'

filt = list(punct + letters + numbers)

In [8]:
doc_stats = []
bad_words = Counter()
good_words = Counter()
for ind, doc in enumerate(tokenized):
    if ind % 10000 == 0:
        print(ind, 'docs processed')
    size = len(doc)
    bad_n = 0
    good_n = 0
    for word in doc:
        bad = False
        for symb in word:
            if symb not in filt:
                bad = True
                bad_words.update([word])
                bad_n += 1
                break
        if not bad:
            good_words.update([word])
            good_n += 1
    doc_stats.append((good_n, bad_n, ind))

0 docs processed
10000 docs processed
20000 docs processed
30000 docs processed
40000 docs processed
50000 docs processed
60000 docs processed
70000 docs processed
80000 docs processed
90000 docs processed
100000 docs processed
110000 docs processed
120000 docs processed
130000 docs processed
140000 docs processed
150000 docs processed
160000 docs processed
170000 docs processed
180000 docs processed
190000 docs processed


In [9]:
# დავასორტიროთ კარგ სიტყვებს - ცუდი სიტყვების მიხედვით
results = [(stat[0]-stat[1], stat[2]) for stat in doc_stats]
results.sort(key=lambda tup:tup[0], reverse=True)
results = results[:50000]

In [10]:
indices = [ind for score, ind in results]
articles = [docs[ind] for ind in indices]

In [11]:
# ავრიოთ და გავყოთ ორ ნაწილად
random.shuffle(articles)
train = articles[:45000]
validation = articles[45000:]

In [12]:
# replace ფუნქციები იმისთვის გვაქვს, რომ ტოკენიზატორი ვერ ითვალისწინებს რამდენიმე შემთხვევას
filtered_train = []
for text in train:
    text = text.replace('"', ' " ')
    text = text.replace('”', ' " ')
    text = text.replace('“', ' " ')
    tokenized = tokenize(text)
    filtered_train.append(tokenized)

filtered_validation = []
for text in validation:
    text = text.replace('"', ' " ')
    text = text.replace('”', ' " ')
    text = text.replace('“', ' " ')
    tokenized = tokenize(text)
    filtered_validation.append(tokenized)

train = filtered_train
validation = filtered_validation


საქმე იმაშია, რომ გვაქვს ძალიან ბევრი უნიკალური ტოკენი და უმეტესობის შეხვედრის სიხშირე არის საკმაოდ ცოტა.
გვიწევს ყველაზე ხშირი ტოკენების გადანახვა და დანარჩენების unk ტოკენებით შეცვლა. 
პრობლემა იმაშია, რომ ამ მიდგომამ არ გაამართლა. იმდენად ბევრი unk ტოკენი ხვდებოდა შემდგომ მოდელს, რომ პრიორიტეტს სწორედ ამ ტოკენს ანიჭებდა.
ამის საწინააღმდეგოდ, თუ vocab-ში ბევრ სიტყვას შევიყვანთ და ამით შევამცირებთ unk ტოკენების რაოდენობას, 
მაშინ მოდელი კომპლექსური გახდება და ამავდროულად overfitting-ზე გავა, რადგან მონაცემები ცოტაა და სიტყვების დიდი ნაწილი სულ 5-10ჯერ იქნება ალბათ კორპუსში.


იმისთვის, რომ მდგომარეობა შეგვემსუბუქებინა, გადავწყვიტეთ შემოგვეღო ბევრნაირი unk ტოკენი. აღვწერ თითოეულს:

NUM_SML - ციფრები

NUM_MED - რიცხვები 10დან 2100-მდე (კორპუსიდან გამომდინარე, ძირითადად იტორიული წლები შედის აქ)

NUM_LRG - 2100დან ზემოთ

SYMB_UNK - სიმბოლო, რომელიც არ შედის ზემოთ შედგენილ სიმბოლოების სიაში

FOREIGN_UNK - უცხოური სიტყვა

GEO_UNK - ქართული სიტყვა, რომელიც VOCAB-ში არაა


In [13]:
def filter_tokens(tokens):
    filtered = []
    for token in tokens:
        if len(token) == 1:
            if token in numbers:
                filtered.append('NUM_SML')
            elif token not in punct and token not in letters:
                filtered.append('SYMB_UNK')
            else:
                filtered.append(token)
        elif token.isdigit():
            try:
                num = float(token)
                if num < 2100:
                    filtered.append('NUM_MED')
                else:
                    filtered.append('NUM_LRG')
            except:
                filtered.append('NUM_MED')
        else:
            foreign = False
            for sm in token:
                if sm not in letters:
                    foreign = True
                    break
            if foreign:
                filtered.append('FOREIGN_UNK')
            else:
                filtered.append(token)
    return filtered


In [14]:
cnt = Counter()
validation = [filter_tokens(doc) for doc in validation]
train = [filter_tokens(doc) for doc in train]
for doc in train:
    cnt.update(doc)

In [15]:
vocab = ['GEO_UNK'] + [word for word, freq in cnt.most_common()[:90000]]
vocab = sorted(vocab)

In [16]:
import time
vocab_set = set(vocab)
def add_geo_unk_words(tokens):
    filtered = []
    for word in tokens:
        if word in vocab_set:
            filtered.append(word)
        else:
            filtered.append('GEO_UNK')
    return filtered

print('adding unk words to train')
train = [add_geo_unk_words(doc) for doc in train]
print('adding unk words to validation')
validation = [add_geo_unk_words(doc) for doc in validation]

adding unk words to train
adding unk words to validation


In [17]:
import json, os

if not os.path.exists('../data/vocab'):
    os.makedirs('../data/vocab')
    
with open('../data/vocab/vocab.json', 'w') as f:
    json.dump(vocab, f, ensure_ascii=False)
    
if not os.path.exists('../data/train'):
    os.makedirs('../data/train')
    
with open('../data/train/train.json', 'w') as f:
    json.dump(train, f, ensure_ascii=False)
    
if not os.path.exists('../data/validation'):
    os.makedirs('../data/validation')

with open('../data/validation/validation.json', 'w') as f:
    json.dump(validation, f, ensure_ascii=False)
